# TEXT2SQL with transformers

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import transformers
print(f"PyTroch Version: {torch.__version__}")
print(f"Transfomers Version: {transformers.__version__}")

PyTroch Version: 1.8.1
Transfomers Version: 4.6.1


In [3]:
from typing import Tuple, Dict, List, Union, Any

## Data to build

In [10]:
from pathlib import Path
import re
import records

schema_re = re.compile(r'\((.+)\)')
num_re = re.compile(r'[-+]?\d*\.\d+|\d+')

db_path = Path("./private")
db = records.Database(f"sqlite:///{db_path / 'samsung_new.db'}")

table_id = "receipts"
table_info = db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql
schema_str = schema_re.findall(table_info.replace("\n", ""))[0]
schema = {}
for tup in schema_str.split(', '):
    c, t = tup.split()
    schema[c.strip('"')] = t

In [11]:
# sqlite can have NULL, INTEGER, REAL, TEXT, BLOB data type
schema

{'index': 'INTEGER',
 'rcept_no': 'TEXT',
 'reprt_code': 'TEXT',
 'bsns_year': 'INTEGER',
 'corp_code': 'TEXT',
 'stock_code': 'TEXT',
 'fs_div': 'TEXT',
 'fs_nm': 'TEXT',
 'sj_div': 'TEXT',
 'sj_nm': 'TEXT',
 'account_nm': 'TEXT',
 'thstrm_nm': 'TEXT',
 'thstrm_dt': 'TEXT',
 'thstrm_amount': 'INTEGER',
 'frmtrm_nm': 'TEXT',
 'frmtrm_dt': 'TEXT',
 'frmtrm_amount': 'INTEGER',
 'bfefrmtrm_nm': 'TEXT',
 'bfefrmtrm_dt': 'TEXT',
 'bfefrmtrm_amount': 'INTEGER'}

In [12]:
set(schema.values())

{'INTEGER', 'TEXT'}

In [13]:
question = "제 51 기에 삼성전자의 유동자산은 어떻게 돼?"
cls_token = "[CLS]"
table_token = "[T]"
column_token = "[C]"

f"{cls_token} {question} [T] {table_id} " + " ".join([f"{column_token} {col} [{typ}]" for col, typ in schema.items()])

'[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼? [T] receipts [C] index [INTEGER] [C] rcept_no [TEXT] [C] reprt_code [TEXT] [C] bsns_year [INTEGER] [C] corp_code [TEXT] [C] stock_code [TEXT] [C] fs_div [TEXT] [C] fs_nm [TEXT] [C] sj_div [TEXT] [C] sj_nm [TEXT] [C] account_nm [TEXT] [C] thstrm_nm [TEXT] [C] thstrm_dt [TEXT] [C] thstrm_amount [INTEGER] [C] frmtrm_nm [TEXT] [C] frmtrm_dt [TEXT] [C] frmtrm_amount [INTEGER] [C] bfefrmtrm_nm [TEXT] [C] bfefrmtrm_dt [TEXT] [C] bfefrmtrm_amount [INTEGER]'

## Build a cumstom Tokenizer

https://huggingface.co/docs/tokenizers/python/latest/pipeline.html

### Normalization

- `normalizers`는 raw text를 더 깨끗하게 만드는 과정이다.
- `NFD` 사용하게 되면 한글은 자음 모음으로 분리된다.
    - NFD(Normalization Form Canonical Decomposition) = 조합형
    - NFC(Normalizaiton Form Canonical Compostion) = 완성형

In [7]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

normalizer = normalizers.Sequence([NFD(), StripAccents()])
# 일반 출력시 합쳐져서 보이지만, for문을 사용하면 분리된다. 
print(normalizer.normalize_str(question))
print("/".join([x for x in normalizer.normalize_str(question)]))

제 51 기에 삼성전자의 유동자산은 어떻게 돼?
ᄌ/ᅦ/ /5/1/ /ᄀ/ᅵ/ᄋ/ᅦ/ /ᄉ/ᅡ/ᆷ/ᄉ/ᅥ/ᆼ/ᄌ/ᅥ/ᆫ/ᄌ/ᅡ/ᄋ/ᅴ/ /ᄋ/ᅲ/ᄃ/ᅩ/ᆼ/ᄌ/ᅡ/ᄉ/ᅡ/ᆫ/ᄋ/ᅳ/ᆫ/ /ᄋ/ᅥ/ᄄ/ᅥ/ᇂ/ᄀ/ᅦ/ /ᄃ/ᅫ/?


### Pre-Tokenization

- `pre_tokenizers`는 텍스트를 더 작은 토큰으로 분리하는 과정이다.
- `Whitespace`는 토큰을 공백을 기준으로 나누면 string의 위치를 같이 반환한다. (start + end) - 기준 regex: `\w+|[^\w\s]+`
- `Punctuation`은 문장 부호를 각각 분리한다. 
- `Digits`를 통해 숫자를 각각 분리할지 말지 결정할 수 있다. 

In [8]:
from tokenizers import pre_tokenizers
from tokenizers.pre_tokenizers import Whitespace, Punctuation, Digits

print("Whitespace: ")
print(Whitespace().pre_tokenize_str(question))

print("Digits individual: False")
pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Digits(individual_digits=False)])
print(pre_tokenizer.pre_tokenize_str(question + "????"))

print("Punctuation + Individual Digits")
pre_tokenizer = pre_tokenizers.Sequence([Whitespace(), Punctuation(), Digits(individual_digits=True)])
print(pre_tokenizer.pre_tokenize_str(question + "????"))

Whitespace: 
[('제', (0, 1)), ('51', (2, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?', (25, 26))]
Digits individual: False
[('제', (0, 1)), ('51', (2, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?????', (25, 30))]
Punctuation + Individual Digits
[('제', (0, 1)), ('5', (2, 3)), ('1', (3, 4)), ('기에', (5, 7)), ('삼성전자의', (8, 13)), ('유동자산은', (14, 19)), ('어떻게', (20, 23)), ('돼', (24, 25)), ('?', (25, 26)), ('?', (26, 27)), ('?', (27, 28)), ('?', (28, 29)), ('?', (29, 30))]


### The Model

- `BPE`: Byte-Pair Encoding 토큰화, 자주 등장하는 character를 합쳐서 표현하는 알고리즘
- `Unigram`: 확률적으로 최적의 subword 토큰을 결정
- `WordLevel`: 단어 단위의 토큰화
- `WordPiece`: Google WordPiece 토큰화

In [38]:
from tokenizers.models import BPE, Unigram, WordLevel, WordPiece

### Post-Processing

- `processors` 에서 후처리를 할 수 있다.
- `TemplateProcessing`을 이용해 원하는 형태로 토큰을 분리할 수 있다.

In [42]:
from tokenizers.processors import PostProcessor

In [ ]:
from tokenizers.processors import TemplateProcessing

post_processor = TemplateProcessing(
    single="[CLS] $A [T] $B [C] $B",
    pair="[CLS] $A [T] $B:1 [C] $B:1",
    special_tokens=[("[CLS]", 1), ("[SEP]", 2)]#, ("[T]", 3), ("[C]", 4), ("[INTEGER]", 5), ("[REAL]", 6), ("[TEXT]", 7), ("[BLOB]", 8)],
)

---

- https://huggingface.co/docs/tokenizers/python/latest/index.html
- https://huggingface.co/transformers/main_classes/tokenizer.html?highlight=pretrainedtokenizer#transformers.PreTrainedTokenizer

In [14]:
input_str = f"{question} [T] {table_id} " + "".join([f"[SEP]{col}" for col in schema]) 
# " ".join([f"{column_token} {col} [{typ}]" for col, typ in schema.items()])
input_str

'제 51 기에 삼성전자의 유동자산은 어떻게 돼? [T] receipts [SEP]index[SEP]rcept_no[SEP]reprt_code[SEP]bsns_year[SEP]corp_code[SEP]stock_code[SEP]fs_div[SEP]fs_nm[SEP]sj_div[SEP]sj_nm[SEP]account_nm[SEP]thstrm_nm[SEP]thstrm_dt[SEP]thstrm_amount[SEP]frmtrm_nm[SEP]frmtrm_dt[SEP]frmtrm_amount[SEP]bfefrmtrm_nm[SEP]bfefrmtrm_dt[SEP]bfefrmtrm_amount'

In [15]:
batch_qs = ["제 51 기에 삼성전자의 유동자산은 어떻게 돼?", "2020년도 삼성전자의 유동자산은 얼마?"]
table_str = f"[T]{table_id}" + "".join([f"[SEP]{col}" for col in schema]) 
batch_ts = [table_str] * len(batch_qs)

In [16]:
from KoBertTokenizer import KoBertTokenizer

In [17]:
# new_special_tokens = ["[T]", "[C]", "[INTEGER]", "[REAL]", "[TEXT]", "[BLOB]"]
# new_special_tokens = list(map(lambda x: AddedToken(x, single_word=True, normalized=False), new_special_tokens))
# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert', add_special_tokens=True, additional_special_tokens=new_special_tokens)
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

In [18]:
special_tokenes2idx = dict(zip(tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids))
special_tokenes2idx

{}

In [19]:
encode_input = tokenizer(
    batch_qs, batch_ts, 
    max_length=512, padding=True, truncation=True, return_tensors="pt", 
    return_attention_mask=True, 
    return_special_tokens_mask=False, 
)
encode_input.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [20]:
batch_size, max_length = encode_input["input_ids"].size()
question_mask = torch.bitwise_and(encode_input["token_type_ids"] == 0, encode_input["attention_mask"].bool())
question_mask[:, 0] = False  # [CLS] mask out
question_mask[:, -1] = False  # [SEP] mask out

# table_mask = torch.where(
#     (encode_input["input_ids"] == 8002), 
#     torch.ones_like(encode_input["input_ids"], dtype=torch.bool), 
#     torch.zeros_like(encode_input["input_ids"], dtype=torch.bool)
# )
# column_mask = torch.where(
#     (encode_input["input_ids"] == 8003),
#     torch.ones_like(encode_input["input_ids"], dtype=torch.bool), 
#     torch.zeros_like(encode_input["input_ids"], dtype=torch.bool)
# )

In [21]:
for x in encode_input["input_ids"]:
    print(tokenizer.decode(x, skip_special_tokens=False))
    print()

[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼?[SEP] [T]receipts[SEP] index[SEP] rcept_no[SEP] reprt_code[SEP] bsns_year[SEP] corp_code[SEP] stock_code[SEP] fs_div[SEP] fs_nm[SEP] sj_div[SEP] sj_nm[SEP] account_nm[SEP] thstrm_nm[SEP] thstrm_dt[SEP] thstrm_amount[SEP] frmtrm_nm[SEP] frmtrm_dt[SEP] frmtrm_amount[SEP] bfefrmtrm_nm[SEP] bfefrmtrm_dt[SEP] bfefrmtrm_amount[SEP]

[CLS] 2020년도 삼성전자의 유동자산은 얼마?[SEP] [T]receipts[SEP] index[SEP] rcept_no[SEP] reprt_code[SEP] bsns_year[SEP] corp_code[SEP] stock_code[SEP] fs_div[SEP] fs_nm[SEP] sj_div[SEP] sj_nm[SEP] account_nm[SEP] thstrm_nm[SEP] thstrm_dt[SEP] thstrm_amount[SEP] frmtrm_nm[SEP] frmtrm_dt[SEP] frmtrm_amount[SEP] bfefrmtrm_nm[SEP] bfefrmtrm_dt[SEP] bfefrmtrm_amount[SEP][PAD][PAD]



In [22]:
for inputs, type_ids in zip(encode_input["input_ids"], encode_input["token_type_ids"]):
    print("--- type_ids = 0")
    print(tokenizer.decode(inputs[type_ids == 0]).replace("[PAD]", ""))
    print("--- type_ids = 1")
    print(tokenizer.decode(inputs[type_ids == 1]))
    print()

--- type_ids = 0
[CLS] 제 51 기에 삼성전자의 유동자산은 어떻게 돼?[SEP]
--- type_ids = 1
[T]receipts[SEP] index[SEP] rcept_no[SEP] reprt_code[SEP] bsns_year[SEP] corp_code[SEP] stock_code[SEP] fs_div[SEP] fs_nm[SEP] sj_div[SEP] sj_nm[SEP] account_nm[SEP] thstrm_nm[SEP] thstrm_dt[SEP] thstrm_amount[SEP] frmtrm_nm[SEP] frmtrm_dt[SEP] frmtrm_amount[SEP] bfefrmtrm_nm[SEP] bfefrmtrm_dt[SEP] bfefrmtrm_amount[SEP]

--- type_ids = 0
[CLS] 2020년도 삼성전자의 유동자산은 얼마?[SEP]
--- type_ids = 1
[T]receipts[SEP] index[SEP] rcept_no[SEP] reprt_code[SEP] bsns_year[SEP] corp_code[SEP] stock_code[SEP] fs_div[SEP] fs_nm[SEP] sj_div[SEP] sj_nm[SEP] account_nm[SEP] thstrm_nm[SEP] thstrm_dt[SEP] thstrm_amount[SEP] frmtrm_nm[SEP] frmtrm_dt[SEP] frmtrm_amount[SEP] bfefrmtrm_nm[SEP] bfefrmtrm_dt[SEP] bfefrmtrm_amount[SEP]



## Encoder

In [23]:
from transformers import BertModel, BertConfig
model = BertModel.from_pretrained("monologg/kobert")
model.resize_token_embeddings(len(tokenizer))

Embedding(8002, 768, padding_idx=1)

In [52]:
model.config.output_hidden_states = True

In [53]:
encode_output = model(**encode_input)

In [58]:
len(encode_output.hidden_states)

13

## Decoder

<img src="https://drive.google.com/uc?id=1PW9oAXfW-ZI-jxGn5q9O_gzUIZnNYaet" alt="Sqlova Decoder Architecture " width="100%" height="auto">

### Select Column

$$\begin{aligned} 
s(n\vert c) &= D_c^T W E_n \\
p(n\vert c) &= \text{softmax}(s(n\vert c))\\
C_c &= \sum_n p(n \vert c) E_n \\
s_{sc}(c) &= W  \tanh ([WD_c; WC_c]) \\
p_{sc}(c) &= \text{softmax}(s_{sc}(c))
\end{aligned}$$

$E_n$ is LSTM output of $n$-th token of question, $D_c$ is the encoding of header $c$, $C_n$ is context vector of question for given column, $[\cdot ; \cdot]$ is concatenation of two vectors, $p_{sc}(c)$ is probability of generating column c.

In [ ]:
class DecoderAttention(nn.Module):
    r"""Decoder Attention Module"""
    def __init__(self, in_features, out_features):
        r"""
        Attention Module
        $$ f(X) = W_1 \tanh ( W_2 X ) $$
        """
        self.W1 = nn.Linear(in_features, out_features)
        self.W2 = nn.Linear(in_features, out_features)

class SelectDecoder(nn.Module):
    r"""SELECT Decoder"""
    def __init__(self, input_size: int, hidden_size: int, num_layers: int, dropout_ratio:float) -> None:
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_ratio = dropout_ratio
        
        self.lstm_q = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)
        self.lstm_h = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, )

In [301]:
encode_output.last_hidden_state.size()

torch.Size([2, 247, 768])

In [303]:
encode_output.pooler_output.size()

torch.Size([2, 768])

In [1]:
class DBEngine:

    def __init__(self, fdb):
        #fdb = 'data/test.db'
        self.db = records.Database('sqlite:///{}'.format(fdb))

    def execute_query(self, table_id, query, *args, **kwargs):
        return self.execute(table_id, query.sel_index, query.agg_index, query.conditions, *args, **kwargs)

    def execute(self, table_id, select_index, aggregation_index, conditions, lower=True):
        if not table_id.startswith('table'):
            table_id = 'table_{}'.format(table_id.replace('-', '_'))
            
        table_info = self.db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql.replace('\n','')
        schema_str = schema_re.findall(table_info)[0]
        schema = {}
        for tup in schema_str.split(', '):
            c, t = tup.split()
            schema[c] = t
        select = 'col{}'.format(select_index)
        agg = agg_ops[aggregation_index]
        if agg:
            select = '{}({})'.format(agg, select)
        where_clause = []
        where_map = {}
        for col_index, op, val in conditions:
            if lower and (isinstance(val, str) or isinstance(val, str)):
                val = val.lower()
            if schema['col{}'.format(col_index)] == 'real' and not isinstance(val, (int, float)):
                try:
                    # print('!!!!!!value of val is: ', val, 'type is: ', type(val))
                    # val = float(parse_decimal(val)) # somehow it generates error.
                    val = float(parse_decimal(val, locale='en_US'))
                    # print('!!!!!!After: val', val)

                except NumberFormatError as e:
                    try:
                        val = float(num_re.findall(val)[0]) # need to understand and debug this part.
                    except:
                        # Although column is of number, selected one is not number. Do nothing in this case.
                        pass
            where_clause.append('col{} {} :col{}'.format(col_index, cond_ops[op], col_index))
            where_map['col{}'.format(col_index)] = val
        where_str = ''
        if where_clause:
            where_str = 'WHERE ' + ' AND '.join(where_clause)
        query = 'SELECT {} AS result FROM {} {}'.format(select, table_id, where_str)

        #print query
        out = self.db.query(query, **where_map)

        return [o.result for o in out]

    def execute_return_query(self, table_id, select_index, aggregation_index, conditions, lower=True):
        if not table_id.startswith('table'):
            table_id = 'table_{}'.format(table_id.replace('-', '_'))
        table_info = self.db.query('SELECT sql from sqlite_master WHERE tbl_name = :name', name=table_id).all()[0].sql.replace('\n','')
        schema_str = schema_re.findall(table_info)[0]
        schema = {}
        for tup in schema_str.split(', '):
            c, t = tup.split()
            schema[c] = t
        select = 'col{}'.format(select_index)
        agg = agg_ops[aggregation_index]
        if agg:
            select = '{}({})'.format(agg, select)
        where_clause = []
        where_map = {}
        for col_index, op, val in conditions:
            if lower and (isinstance(val, str) or isinstance(val, str)):
                val = val.lower()
            if schema['col{}'.format(col_index)] == 'real' and not isinstance(val, (int, float)):
                try:
                    # print('!!!!!!value of val is: ', val, 'type is: ', type(val))
                    # val = float(parse_decimal(val)) # somehow it generates error.
                    val = float(parse_decimal(val, locale='en_US'))
                    # print('!!!!!!After: val', val)

                except NumberFormatError as e:
                    val = float(num_re.findall(val)[0])
            where_clause.append('col{} {} :col{}'.format(col_index, cond_ops[op], col_index))
            where_map['col{}'.format(col_index)] = val
        where_str = ''
        if where_clause:
            where_str = 'WHERE ' + ' AND '.join(where_clause)
        query = 'SELECT {} AS result FROM {} {}'.format(select, table_id, where_str)

        #print query
        out = self.db.query(query, **where_map)


        return [o.result for o in out], query
    def show_table(self, table_id):
        if not table_id.startswith('table'):
            table_id = 'table_{}'.format(table_id.replace('-', '_'))
        rows = self.db.query('select * from ' +table_id)
        print(rows.dataset)

---

# SQLova Model

In [1]:
from sqlova.model.nl2sql.wikisql_models import *

In [2]:
import json
import torch
from pathlib import Path
from tqdm.notebook import tqdm

In [3]:
from typing import Tuple, Dict, List

In [9]:
def load_data():
    path_sql = Path("NLSQL.jsonl")
    path_table = Path("table.jsonl")

    data = []
    table = {}
    with path_sql.open("r") as f:
        for idx, line in enumerate(f):
            t1 = json.loads(line.strip())
            data.append(t1)

    with path_table.open("r") as f:
        for idx, line in enumerate(f):
            t1 = json.loads(line.strip())
            table[t1['id']] = t1
            
    return data, table

In [10]:
data, table = load_data()

In [11]:
import os

# multiprocessing lib doesn’t have it implemented on Windows
# https://discuss.pytorch.org/t/cant-pickle-local-object-dataloader-init-locals-lambda/31857/14
num_workers = 0 if os.name == "nt" else 4

In [12]:
data_loader = torch.utils.data.DataLoader(
    batch_size=2,
    dataset=data,
    shuffle=True,
    num_workers=num_workers,
    collate_fn=lambda x: x # now dictionary values are not merged!
)

db

In [13]:
from tsv2jsonl import DBEngine

In [14]:
db_path = Path("./private")
dbengine = DBEngine(db_path / "samsung_new.db")

In [180]:
table_schema_lengths = []
for t in dbengine.db.get_table_names():
    dbengine.get_schema_info(t)
    table_schema_lengths.append(len(dbengine.schema))

In [229]:
# engine = DBEngine(db_path / "samsung_new.db")
# engine.execute(table_id="receipts", query)

get models

In [222]:
from KoBertTokenizer import KoBertTokenizer
from transformers import BertModel, BertConfig

def get_bert(model_path: str, device: str, max_col_length: int, output_hidden_states: bool=False):
    
    special_tokens = ["[S]", "[E]", "[COL]"] # sequence start, sequence end, column tokens
    tokenizer = KoBertTokenizer.from_pretrained(model_path, add_special_tokens=True, additional_special_tokens=special_tokens)
    config = BertConfig.from_pretrained(model_path)
    config.output_hidden_states = output_hidden_states
    
    model = BertModel.from_pretrained(model_path)
    model.resize_token_embeddings(len(tokenizer))
    model.config.output_hidden_states = output_hidden_states
    model.to(device)
    
    return model, tokenizer, config

In [223]:
model_path = "monologg/kobert"
device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu" 

agg_ops = ['', 'MAX', 'MIN', 'COUNT', 'SUM', 'AVG']
cond_ops = ['=', '>', '<', 'OP']

num_target_layers = 1  # "The Number of final layers of BERT to be used in downstream task."
# Get BERT
model_bert, tokenizer_bert, config_bert = get_bert(model_path=model_path, device=device, max_col_length=max(table_schema_lengths))

# Get Seq-to-SQL
iS = config_bert.hidden_size * num_target_layers  # Seq-to-SQL input vector dimenstion
hS = 100
lS = 2
dr = 0.3

n_cond_ops = len(cond_ops)
n_agg_ops = len(agg_ops)
model = Seq2SQL_v1(iS, hS, lS, dr, n_cond_ops, n_agg_ops)
model = model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


get parameters

In [224]:
lr = 1e-3
lr_bert = 1e-5

opt = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()),
                               lr=lr, weight_decay=0)
opt_bert = torch.optim.AdamW(filter(lambda p: p.requires_grad, model_bert.parameters()),
                            lr=lr_bert, weight_decay=0)

In [225]:
model.train()
model_bert.train()

ave_loss = 0
cnt = 0  # count the # of examples
cnt_sc = 0  # count the # of correct predictions of select column
cnt_sa = 0  # of selectd aggregation
cnt_wn = 0  # of where number
cnt_wc = 0  # of where column
cnt_wo = 0  # of where operator
cnt_wv = 0  # of where-value
cnt_wvi = 0  # of where-value index (on question tokens)
cnt_lx = 0  # of logical form acc
cnt_x = 0  # of execution acc

In [226]:
st_pos = 0

for iB, data in enumerate(tqdm(data_loader, desc='TRAIN')):
    cnt += len(data)
    
    if cnt < st_pos:
        continue
    break

TRAIN:   0%|          | 0/21120 [00:00<?, ?it/s]

In [186]:
len(data)

2

In [227]:
start_tkn = tokenizer_bert.additional_special_tokens[0]
start_tkn_id = tokenizer_bert.additional_special_tokens_ids[0]
end_tkn = tokenizer_bert.additional_special_tokens[1]
end_tkn_id = tokenizer_bert.additional_special_tokens_ids[1]

batch_qs = [jsonl["question"] for jsonl in data]
tid = [jsonl["table_id"] for jsonl in data]
batch_sqls = [jsonl["sql"] for jsonl in data]
batch_ts = []
for table_id in tid:
    dbengine.get_schema_info(table_id)
    table_str = f"{start_tkn}{table_id}{end_tkn}" + "".join([
        f"[COL]{start_tkn}{col}{end_tkn}" for col in dbengine.schema
    ]) 
    batch_ts.append(table_str)
    
encode_input = tokenizer_bert(
    batch_qs, batch_ts, 
    max_length=512, padding=True, truncation=True, return_tensors="pt", 
    return_attention_mask=True, 
    return_special_tokens_mask=False, 
)

In [228]:
print(tokenizer_bert.decode(encode_input["input_ids"][0]))

[CLS] 삼성전자의 2017 비유동부채가 어떻게 돼?[SEP] [S] receipts [E] [COL] [S] index [E] [COL] [S] rcept_no [E] [COL] [S] reprt_code [E] [COL] [S] bsns_year [E] [COL] [S] corp_code [E] [COL] [S] stock_code [E] [COL] [S] fs_div [E] [COL] [S] fs_nm [E] [COL] [S] sj_div [E] [COL] [S] sj_nm [E] [COL] [S] account_nm [E] [COL] [S] thstrm_nm [E] [COL] [S] thstrm_dt [E] [COL] [S] thstrm_amount [E] [COL] [S] frmtrm_nm [E] [COL] [S] frmtrm_dt [E] [COL] [S] frmtrm_amount [E] [COL] [S] bfefrmtrm_nm [E] [COL] [S] bfefrmtrm_dt [E] [COL] [S] bfefrmtrm_amount [E] [SEP]


In [369]:
def get_answer(input_ids, mask, batch_size, start_tkn_id, end_tkn_id):
    r"""
    answer should include end token: [E]
    """
    masked_input_ids = input_ids[mask]
    start_tkn_mask = masked_input_ids == start_tkn_id
    end_tkn_mask = masked_input_ids == end_tkn_id
    table_col_length = masked_input_ids.view(batch_size, -1).size(1)
    start_end_mask = torch.bitwise_or(start_tkn_mask, end_tkn_mask)
    index = torch.arange(table_col_length).repeat(batch_size)[start_end_mask].view(batch_size, -1, 2)
    tkn_lengths = index[:, :, 1] - index[:, :, 0]
    answer_col_tkns = [x.split(tkn_length.tolist()) for x, tkn_length in zip(
        masked_input_ids[~start_tkn_mask].view(batch_size, -1), tkn_lengths)]
    return answer_col_tkns

# def get_decoder_input_mask(input_ids, mask, batch_size, end_tkn_id):
#     r"""
#     input should only include end token: [S]
#     """
#     end_tkn_mask = input_ids == end_tkn_id
#     end_index = torch.arange(input_ids.size(1)).repeat(batch_size)[end_tkn_mask.view(-1)].view(batch_size, -1)
#     return mask.scatter(1, end_index, False)

def get_decoder_input_mask(input_ids, mask, batch_size, end_tkn_id):
    r"""
    input should only contains word tokens:
    """
    start_tkn_mask = input_ids == start_tkn_id
    end_tkn_mask = input_ids == end_tkn_id
    start_end_mask = torch.bitwise_or(start_tkn_mask, end_tkn_mask)
    index = torch.arange(input_ids.size(1)).repeat(batch_size)[start_end_mask.view(-1)].view(batch_size, -1)
    return mask.scatter(1, index, False)

In [370]:
def get_input_mask_and_answer(encode_input, tokenizer):
    r"""
    table -> database table name(id)
    header -> database header
    
    returns:
        input_question_mask, input_table_mask, input_header_mask, answer_table_tkns, answer_header_tkns
    """
    batch_size, max_length = encode_input["input_ids"].size()
    sep_tkn_mask = encode_input["input_ids"] == tokenizer.sep_token_id
    start_tkn_id, end_tkn_id, col_tkn_id = tokenizer.additional_special_tokens_ids
    
    input_question_mask = torch.bitwise_and(encode_input["token_type_ids"] == 0, encode_input["attention_mask"].bool())
    input_question_mask = torch.bitwise_and(input_question_mask, ~sep_tkn_mask) # [SEP] mask out
    input_question_mask[:, 0] = False  # [CLS] mask out

    db_mask = torch.bitwise_and(encode_input["token_type_ids"] == 1, encode_input["attention_mask"].bool())
    db_mask = torch.bitwise_xor(db_mask, sep_tkn_mask)
    col_tkn_mask = encode_input["input_ids"] == col_tkn_id
    db_mask = torch.bitwise_and(db_mask, ~col_tkn_mask)
    # split table_mask and header_mask
    input_idx = torch.arange(max_length).repeat(batch_size, 1)
    db_idx = input_idx[db_mask]
    table_header_tkn_idx = db_idx[db_idx > 0]
    table_start_idx = table_header_tkn_idx.view(batch_size, -1)[:, 0] + 1
    start_tkn_idx = table_header_tkn_idx[1:][table_header_tkn_idx.diff() == 2].view(batch_size, -1)
    table_end_sep_idx = start_tkn_idx[:, 0] - 1
    split_size = torch.stack([
        table_end_sep_idx-table_start_idx+1, table_header_tkn_idx.view(batch_size, -1).size(1)-(table_end_sep_idx-table_start_idx+1)
    ]).transpose(0, 1)

    # Token idx
    table_tkn_idx, header_tkn_idx = map(
        lambda x: torch.stack(x), 
        zip(*[torch.split(x, size.tolist()) for x, size in zip(table_header_tkn_idx.view(batch_size, -1), split_size)])
    )

    table_tkn_idx = table_tkn_idx[:, 1:]
    # Mask include [S] & [E] tokens
    table_tkn_mask = torch.zeros_like(encode_input["input_ids"], dtype=torch.bool).scatter(1, table_tkn_idx, True)
    header_tkn_mask = torch.zeros_like(encode_input["input_ids"], dtype=torch.bool).scatter(1, header_tkn_idx, True)

    # For Answer, Maskout [S] for table & header 
    answer_table_tkns = get_answer(
        encode_input["input_ids"], table_tkn_mask, batch_size, start_tkn_id, end_tkn_id
    )
    answer_header_tkns = get_answer(
        encode_input["input_ids"], header_tkn_mask, batch_size, start_tkn_id, end_tkn_id
    )

    # For Decoder Input, Maskout [S], [E] for table & header  
    input_table_mask = get_decoder_input_mask(
        encode_input["input_ids"], table_tkn_mask, batch_size, end_tkn_id
    )
    input_header_mask = get_decoder_input_mask(
        encode_input["input_ids"], header_tkn_mask, batch_size, end_tkn_id
    )
    
    # [COL] token mask: this is for attention
    col_tkn_idx = input_idx[col_tkn_mask].view(batch_size, -1)
    input_col_mask = torch.zeros_like(encode_input["input_ids"], dtype=torch.bool).scatter(1, col_tkn_idx, True)
    
    return input_question_mask, input_table_mask, input_header_mask, input_col_mask, answer_table_tkns, answer_header_tkns

In [372]:
input_question_mask, input_table_mask, input_header_mask, input_col_mask, answer_table_tkns, answer_header_tkns \
    = get_input_mask_and_answer(encode_input, tokenizer_bert)

In [373]:
print(tokenizer_bert.decode(encode_input["input_ids"][input_question_mask]))
print(tokenizer_bert.decode(encode_input["input_ids"][input_table_mask]))
print(tokenizer_bert.decode(encode_input["input_ids"][input_header_mask]))
print(tokenizer_bert.decode(encode_input["input_ids"][input_col_mask]))
print("---answer table tokens---")
print(answer_table_tkns)
print("---answer column tokens---")
for b in answer_col_tkns:
    print("--- batch ---")
    for x in b:
        print(x)

삼성전자의 2017 비유동부채가 어떻게 돼? 삼성전자 49기 유동자산이 몇이야?
receipts receipts
index rcept_no reprt_code bsns_year corp_code stock_code fs_div fs_nm sj_div sj_nm account_nm thstrm_nm thstrm_dt thstrm_amount frmtrm_nm frmtrm_dt frmtrm_amount bfefrmtrm_nm bfefrmtrm_dt bfefrmtrm_amount index rcept_no reprt_code bsns_year corp_code stock_code fs_div fs_nm sj_div sj_nm account_nm thstrm_nm thstrm_dt thstrm_amount frmtrm_nm frmtrm_dt frmtrm_amount bfefrmtrm_nm bfefrmtrm_dt bfefrmtrm_amount
[COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL] [COL]
---answer table tokens---
[(tensor([ 517,  437,  382,  389,  405,  432,  442,  440, 8003]),), (tensor([ 517,  437,  382,  389,  405,  432,  442,  440, 8003]),)]
---answer column tokens---
--- batch ---
tensor([517, 409, 388, 389, 457])
tensor([517, 435, 382, 389, 432, 442, 365, 425, 427

In [374]:
encode_output = model_bert(**encode_input)

In [375]:
encode_output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [376]:
def pad(batches: Tuple[torch.Tensor], lengths: List[int], model: BertModel, pad_idx: int=1) -> torch.Tensor:
    padded = []
    max_length = max(lengths)
    for x in batches:
        if len(x) < max_length:
            pad_tensor = model.embeddings.word_embeddings(torch.LongTensor([pad_idx]*(max_length - len(x))))
            padded.append(torch.cat([x, pad_tensor]))
        else:
            padded.append(x)
    return torch.stack(padded)

def get_decoder_batches(encode_output, mask, model, pad_idx):
    lengths = mask.sum(1)
    tensors = encode_output.last_hidden_state[mask, :]
    batches = torch.split(tensors, lengths.tolist())
    if lengths.ne(lengths.max()).sum().item() != 0:
        # pad not same length tokens
        tensors_padded = pad(batches, lengths.tolist(), model, pad_idx=pad_idx)
    else:
        # just stack the splitted tensors
        tensors_padded = torch.stack(batches)
    return tensors_padded, lengths.tolist()

In [377]:
question_padded, question_lengths = get_decoder_batches(encode_output, input_question_mask, model_bert, pad_idx=tokenizer_bert.pad_token_id)
table_padded, table_lengths = get_decoder_batches(encode_output, input_table_mask, model_bert, pad_idx=tokenizer_bert.pad_token_id)
header_padded, header_lengths = get_decoder_batches(encode_output, input_header_mask, model_bert, pad_idx=tokenizer_bert.pad_token_id)
col_padded, col_lengths = get_decoder_batches(encode_output, input_col_mask, model_bert, pad_idx=tokenizer_bert.pad_token_id)

In [378]:
question_padded.size(), table_padded.size(), header_padded.size(), col_padded.size()

(torch.Size([2, 12, 768]),
 torch.Size([2, 8, 768]),
 torch.Size([2, 181, 768]),
 torch.Size([2, 20, 768]))

In [125]:
def get_g(sql_i):
    """
    for backward compatibility, separated with get_g
    
    sc: select column
    sa: select agg
    wn: where number
    wc: where column
    wo: where operator
    wv: where value
    """
    g_sc = []
    g_sa = []
    g_wn = []
    g_wc = []
    g_wo = []
    g_wv = []
    for b, psql_i1 in enumerate(sql_i):
        g_sc.append( psql_i1["sel"] )
        g_sa.append( psql_i1["agg"])

        conds = psql_i1['conds']
        if not psql_i1["agg"] < 0:
            g_wn.append( len( conds ) )
            g_wc.append( get_wc1(conds) )
            g_wo.append( get_wo1(conds) )
            g_wv.append( get_wv1(conds) )
        else:
            raise EnvironmentError
    return g_sc, g_sa, g_wn, g_wc, g_wo, g_wv

In [126]:
g_sc, g_sa, g_wn, g_wc, g_wo, g_wv = get_g(batch_sqls)


In [424]:
g_sa

[0, 0]

In [418]:
class ColumnAttention(nn.Module):
    r"""Decoder Column Attention Module"""
    def __init__(self, in_features, out_features):
        super().__init__()
        r"""
        Attention Module
        $$ S(a, b) = a x ( W x b ) $$
        """
        self.linear = nn.Linear(in_features, out_features)
        self.softmax = nn.Softmax(dim=2)
        
    def forward(self, o_q, o_c):
        r"""
        Calculate for each column tokens, How much related to question tokens?
        o_q: LSTM output of question 
        o_c: LSTM output of column
        
        return context atttended to question tokens
        """
        o_q_transform = self.linear(o_q)  # (B, T_q, H)
        s_col2question = torch.bmm(o_c, o_q_transform.transpose(1, 2))  # (B, T_c, H) x (B, H, T_q) = (B, T_c, T_q)
        p_col2question = self.softmax(s_col2question)
        
        # p_col2question: (B, T_c, T_q) -> (B, T_c, T_q, 1)
        # o_q: (B, 1, T_q, H)
        # p_col2question \odot o_q = (B, T_c, T_q, 1) \odot (B, 1, T_q, H) = (B, T_c, T_q, H)
        # -> reduce sum to T_q to get context for each column (B, T_c, H)
        context = torch.mul(p_col2question.unsqueeze(3), o_q.unsqueeze(1)).sum(dim=2)
        return context

class SelectDecoder(nn.Module):
    r"""SELECT Decoder"""
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_layers: int=2, dropout_ratio:float=0.3) -> None:
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_ratio = dropout_ratio
        
        self.lstm_q = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)
        self.lstm_h = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)
        
        self.col2question_attn = ColumnAttention(hidden_size, hidden_size)
        self.output_layer = nn.Sequential(
            nn.Linear(2*hidden_size, 2*hidden_size),
            nn.Tanh(),
            nn.Linear(2*hidden_size, output_size)
        )

    def forward(self, question_padded, table_padded, header_padded, col_padded):
        r"""
        predict column index
        """
        o_q, (h_q, c_q) = self.lstm_q(question_padded)  # o_q: (B, T_q, H)
        o_c, (h_c, c_c) = self.lstm_h(col_padded)  # o_c: (B, T_c, H)
        o_h, (h_h, c_h) = self.lstm_h(header_padded)  # h_h: (n_direc*num_layers, B, H/2)
        
        context = self.col2question_attn(o_q, o_c)  # (B, T_c, H)
        header_summary = torch.cat([h for h in h_h[-2:]], dim=1).unsqueeze(1).repeat(1, context.size(1), 1) # (B, T_C, H)
        vec = torch.cat([context, header_summary], dim=2)
        output = self.output_layer(vec)
        return output.squeeze(-1)
    

class AggDecoder(nn.Module):
    r"""AGG Decoder"""
    def __init__(self, input_size: int, hidden_size: int, output_size: int, num_layers: int=2, dropout_ratio:float=0.3) -> None:
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_ratio = dropout_ratio
        
        self.lstm_q = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)
        self.lstm_h = nn.LSTM(input_size, int(hidden_size / 2), num_layers, dropout=dropout_ratio, batch_first=True, bidirectional=True)
        
        self.col2question_attn = ColumnAttention(hidden_size, hidden_size)
        self.output_layer = nn.Sequential(
            nn.Linear(2*hidden_size, 2*hidden_size),
            nn.Tanh(),
            nn.Linear(2*hidden_size, output_size)
        )

    def forward(self, question_padded, table_padded, header_padded, col_padded, select_output):
        r"""
        predict agg index
        """
        select_prob = torch.softmax(select_output, 1) # (B, T_c)
        
        o_q, (h_q, c_q) = self.lstm_q(question_padded)  # o_q: (B, T_q, H)
        o_c, (h_c, c_c) = self.lstm_h(col_padded)  # o_c: (B, T_c, H)
        o_h, (h_h, c_h) = self.lstm_h(header_padded)  # h_h: (n_direc*num_layers, B, H/2)
        
        context = self.col2question_attn(o_q, o_c)  # (B, T_c, H)
        header_summary = torch.cat([h for h in h_h[-2:]], dim=1).unsqueeze(1).repeat(1, context.size(1), 1) # (B, T_C, H)
        vec = torch.cat([context, header_summary], dim=2)
        output = self.output_layer(vec)
        return output.squeeze(-1)

In [380]:
input_size = config_bert.hidden_size
hidden_size = 100
num_layers = 2
dropout_ratio = 0.3
output_size = 1

In [421]:
select_decoder = SelectDecoder(input_size, hidden_size, output_size=1, num_layers=num_layers, dropout_ratio=dropout_ratio)

In [422]:
select_output = select_decoder(question_padded, table_padded, header_padded, col_padded)

In [434]:
select_output

tensor([[-0.0040, -0.0039, -0.0040, -0.0039, -0.0038, -0.0039, -0.0039, -0.0040,
         -0.0041, -0.0040, -0.0041, -0.0039, -0.0039, -0.0040, -0.0041, -0.0041,
         -0.0041, -0.0042, -0.0041, -0.0044],
        [-0.0380, -0.0380, -0.0380, -0.0379, -0.0379, -0.0379, -0.0379, -0.0380,
         -0.0380, -0.0379, -0.0379, -0.0379, -0.0380, -0.0380, -0.0380, -0.0380,
         -0.0380, -0.0380, -0.0380, -0.0380]], grad_fn=<SqueezeBackward1>)

In [435]:
torch.softmax(select_output, 1)

tensor([[0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
         0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
         0.0500, 0.0500],
        [0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
         0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500, 0.0500,
         0.0500, 0.0500]], grad_fn=<SoftmaxBackward>)

In [423]:
select_output.size()

torch.Size([2, 20])